# DQN for continuous action space


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('MountainCarContinuous-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [4]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [5]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [6]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [7]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.339260107412202 -2.8400300900230797
actions: 1 0
rewards: 1.0 1.0


In [8]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [9]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [11]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [12]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [13]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float64, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float64, [None], name='targetQs')
    return states, actions, targetQs

In [14]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        #predictions = tf.nn.softmax(logits)

        # return actions logits
        return logits

In [15]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [20]:
def model_loss(action_size, hidden_size, states, actions, targetQs):
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    Qs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states) # nextQs
    targetQs = tf.reshape(targetQs, shape=[-1, 1])
    gloss = tf.reduce_mean(neg_log_prob_actions * targetQs) # DPG
    gloss += tf.reduce_mean(neg_log_prob_actions * Qs) # DPG
    dloss = tf.reduce_mean(tf.square(Qs - targetQs)) # DQN
    # dloss = tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs,
    #                                                 labels=tf.nn.sigmoid(targetQs))
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, reuse=True) # Qs
    gloss += tf.reduce_mean(neg_log_prob_actions * dQs) # DPG
    dloss += tf.reduce_mean(tf.square(dQs - targetQs)) # DQN
    # dloss += tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs,
    #                                                  labels=tf.nn.sigmoid(targetQs))
    gloss1 = tf.reduce_mean(neg_log_prob_actions)
    gloss2 = tf.reduce_mean(Qs)
    gloss3 = tf.reduce_mean(dQs)
    gloss4 = tf.reduce_mean(targetQs)
    return actions_logits, Qs, gloss, dloss, gloss1, gloss2, gloss3, gloss4

In [21]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [22]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_loss, self.d_loss, self.g_loss1, self.g_loss2, self.g_loss3, self.g_loss4 = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, targetQs=self.targetQs) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

In [23]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, 
                               replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [24]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111,)
action size:2


In [ ]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
action_size = 2
state_size = 4
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
memory_size = 50000            # memory capacity
batch_size = 500                # experience mini-batch size
gamma = 0.99                   # future reward discount

In [ ]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [ ]:
state = env.reset()
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done is True:
        state = env.reset()

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list, gloss_list, dloss_list = [], [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        gloss_batch, dloss_batch = [], []
        gloss1_batch, gloss2_batch, gloss3_batch, gloss4_batch = [], [], [], []
        state = env.reset()

        # Training steps/batches
        for num_steps in range(1111111111):
            # Explore (Env) or Exploit (Model)
            total_step += 0.001
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            total_reward += reward
            state = next_state

            # Training
            batch = memory.sample(batch_size)
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            nextQs_logits = sess.run(model.Qs_logits, feed_dict = {model.states: next_states})
            #nextQs = np.max(nextQs_logits, axis=1) * (1-dones)
            nextQs = nextQs_logits.reshape([-1]) * (1-dones)
            targetQs = rewards + (gamma * nextQs)
            gloss, dloss, gloss1, gloss2, gloss3, gloss4, _, _ = sess.run([model.g_loss, model.d_loss,
                                                                           model.g_loss1, model.g_loss2, 
                                                                           model.g_loss3, model.g_loss4,
                                                                           model.g_opt, model.d_opt],
                                                                          feed_dict = {model.states: states, 
                                                                                       model.actions: actions,
                                                                                       model.targetQs: targetQs})
            gloss_batch.append(gloss)
            dloss_batch.append(dloss)
            gloss1_batch.append(gloss1)
            gloss2_batch.append(gloss2)
            gloss3_batch.append(gloss3)
            gloss4_batch.append(gloss4)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'Steps:{}'.format(num_steps),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'gloss:{:.4f}'.format(np.mean(gloss_batch)),
              'gloss1-lgP:{:.4f}'.format(np.mean(gloss1_batch)), #-logp
              'gloss2-gQs:{:.4f}'.format(np.mean(gloss2_batch)),#gQs
              'gloss3-dQs:{:.4f}'.format(np.mean(gloss3_batch)),#dQs
              'gloss4-tgtQ:{:.4f}'.format(np.mean(gloss4_batch)),#tgtQs
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        gloss_list.append([ep, np.mean(gloss_batch)])
        dloss_list.append([ep, np.mean(dloss_batch)])
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.        
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 Steps:14 meanR:15.0000 R:15.0000 gloss:0.6979 gloss1-lgP:0.6977 gloss2-gQs:0.0376 gloss3-dQs:-0.0733 gloss4-tgtQ:1.0360 dloss:2.2325 exploreP:1.0000
Episode:1 Steps:11 meanR:13.5000 R:12.0000 gloss:0.7891 gloss1-lgP:0.6952 gloss2-gQs:0.0761 gloss3-dQs:-0.0152 gloss4-tgtQ:1.0744 dloss:2.1906 exploreP:1.0000
Episode:2 Steps:28 meanR:18.6667 R:29.0000 gloss:0.9428 gloss1-lgP:0.6947 gloss2-gQs:0.1420 gloss3-dQs:0.0781 gloss4-tgtQ:1.1371 dloss:2.1265 exploreP:1.0000
Episode:3 Steps:10 meanR:16.7500 R:11.0000 gloss:1.0776 gloss1-lgP:0.6927 gloss2-gQs:0.1994 gloss3-dQs:0.1612 gloss4-tgtQ:1.1950 dloss:2.0814 exploreP:1.0000
Episode:4 Steps:14 meanR:16.4000 R:15.0000 gloss:1.1848 gloss1-lgP:0.6936 gloss2-gQs:0.2446 gloss3-dQs:0.2270 gloss4-tgtQ:1.2366 dloss:2.0345 exploreP:1.0000
Episode:5 Steps:19 meanR:17.0000 R:20.0000 gloss:1.3333 gloss1-lgP:0.6938 gloss2-gQs:0.3100 gloss3-dQs:0.3136 gloss4-tgtQ:1.2982 dloss:1.9888 exploreP:1.0000
Episode:6 Steps:100 meanR:29.0000 R:101.0000 gloss

Episode:53 Steps:24 meanR:22.2222 R:25.0000 gloss:26.0957 gloss1-lgP:0.6928 gloss2-gQs:12.3962 gloss3-dQs:12.6896 gloss4-tgtQ:12.5813 dloss:14.5889 exploreP:0.9999
Episode:54 Steps:14 meanR:22.0909 R:15.0000 gloss:26.3004 gloss1-lgP:0.6926 gloss2-gQs:12.4983 gloss3-dQs:12.7365 gloss4-tgtQ:12.7409 dloss:12.7895 exploreP:0.9999
Episode:55 Steps:35 meanR:22.3393 R:36.0000 gloss:26.5550 gloss1-lgP:0.6923 gloss2-gQs:12.5770 gloss3-dQs:12.9551 gloss4-tgtQ:12.8259 dloss:12.5972 exploreP:0.9999
Episode:56 Steps:11 meanR:22.1579 R:12.0000 gloss:26.5400 gloss1-lgP:0.6916 gloss2-gQs:12.6016 gloss3-dQs:13.0340 gloss4-tgtQ:12.7384 dloss:14.9610 exploreP:0.9999
Episode:57 Steps:15 meanR:22.0517 R:16.0000 gloss:26.2433 gloss1-lgP:0.6927 gloss2-gQs:12.4313 gloss3-dQs:12.8532 gloss4-tgtQ:12.6029 dloss:13.4149 exploreP:0.9999
Episode:58 Steps:17 meanR:21.9831 R:18.0000 gloss:26.0254 gloss1-lgP:0.6923 gloss2-gQs:12.2926 gloss3-dQs:12.7342 gloss4-tgtQ:12.5639 dloss:11.7371 exploreP:0.9999
Episode:59 Steps

Episode:104 Steps:27 meanR:22.0000 R:28.0000 gloss:28.6366 gloss1-lgP:0.6919 gloss2-gQs:13.7490 gloss3-dQs:13.9031 gloss4-tgtQ:13.7332 dloss:10.3301 exploreP:0.9998
Episode:105 Steps:10 meanR:21.9100 R:11.0000 gloss:28.2950 gloss1-lgP:0.6930 gloss2-gQs:13.7027 gloss3-dQs:13.4401 gloss4-tgtQ:13.6857 dloss:10.3607 exploreP:0.9998
Episode:106 Steps:42 meanR:21.3300 R:43.0000 gloss:28.5463 gloss1-lgP:0.6932 gloss2-gQs:13.6805 gloss3-dQs:13.7873 gloss4-tgtQ:13.7154 dloss:9.4510 exploreP:0.9998
Episode:107 Steps:17 meanR:21.2500 R:18.0000 gloss:28.0100 gloss1-lgP:0.6917 gloss2-gQs:13.5151 gloss3-dQs:13.5313 gloss4-tgtQ:13.4475 dloss:10.1294 exploreP:0.9998
Episode:108 Steps:20 meanR:21.3000 R:21.0000 gloss:28.0017 gloss1-lgP:0.6937 gloss2-gQs:13.4419 gloss3-dQs:13.3802 gloss4-tgtQ:13.5422 dloss:9.5357 exploreP:0.9998
Episode:109 Steps:30 meanR:21.5100 R:31.0000 gloss:30.4013 gloss1-lgP:0.6927 gloss2-gQs:14.6800 gloss3-dQs:14.5188 gloss4-tgtQ:14.6904 dloss:9.5128 exploreP:0.9998
Episode:110 S

Episode:154 Steps:34 meanR:23.0400 R:35.0000 gloss:27.2543 gloss1-lgP:0.6940 gloss2-gQs:13.0343 gloss3-dQs:13.1185 gloss4-tgtQ:13.1158 dloss:7.5426 exploreP:0.9997
Episode:155 Steps:21 meanR:22.9000 R:22.0000 gloss:26.5372 gloss1-lgP:0.6927 gloss2-gQs:12.5898 gloss3-dQs:13.0568 gloss4-tgtQ:12.6635 dloss:7.8645 exploreP:0.9996
Episode:156 Steps:30 meanR:23.0900 R:31.0000 gloss:26.2079 gloss1-lgP:0.6949 gloss2-gQs:12.4303 gloss3-dQs:12.7454 gloss4-tgtQ:12.5422 dloss:7.6166 exploreP:0.9996
Episode:157 Steps:32 meanR:23.2600 R:33.0000 gloss:27.1258 gloss1-lgP:0.6940 gloss2-gQs:12.9587 gloss3-dQs:13.0839 gloss4-tgtQ:13.0400 dloss:8.6485 exploreP:0.9996
Episode:158 Steps:28 meanR:23.3700 R:29.0000 gloss:26.3424 gloss1-lgP:0.6937 gloss2-gQs:12.6629 gloss3-dQs:12.5482 gloss4-tgtQ:12.7595 dloss:8.3322 exploreP:0.9996
Episode:159 Steps:20 meanR:23.4300 R:21.0000 gloss:27.0947 gloss1-lgP:0.6913 gloss2-gQs:12.9937 gloss3-dQs:13.1168 gloss4-tgtQ:13.0822 dloss:6.4355 exploreP:0.9996
Episode:160 Step

Episode:204 Steps:15 meanR:23.4800 R:16.0000 gloss:26.7042 gloss1-lgP:0.6952 gloss2-gQs:12.6311 gloss3-dQs:13.0528 gloss4-tgtQ:12.7317 dloss:7.4369 exploreP:0.9995
Episode:205 Steps:13 meanR:23.5100 R:14.0000 gloss:27.4754 gloss1-lgP:0.6940 gloss2-gQs:13.2545 gloss3-dQs:12.9642 gloss4-tgtQ:13.3669 dloss:7.5591 exploreP:0.9995
Episode:206 Steps:17 meanR:23.2600 R:18.0000 gloss:28.1931 gloss1-lgP:0.6935 gloss2-gQs:13.5030 gloss3-dQs:13.5426 gloss4-tgtQ:13.6038 dloss:7.8438 exploreP:0.9995
Episode:207 Steps:37 meanR:23.4600 R:38.0000 gloss:28.2940 gloss1-lgP:0.6950 gloss2-gQs:13.4674 gloss3-dQs:13.6958 gloss4-tgtQ:13.5442 dloss:8.3638 exploreP:0.9995
Episode:208 Steps:18 meanR:23.4400 R:19.0000 gloss:27.9548 gloss1-lgP:0.6942 gloss2-gQs:13.4285 gloss3-dQs:13.3984 gloss4-tgtQ:13.4429 dloss:8.0691 exploreP:0.9995
Episode:209 Steps:26 meanR:23.4000 R:27.0000 gloss:27.7849 gloss1-lgP:0.6951 gloss2-gQs:13.1566 gloss3-dQs:13.5397 gloss4-tgtQ:13.2741 dloss:7.4381 exploreP:0.9995
Episode:210 Step

...

Episode:2724 Steps:32 meanR:23.4300 R:33.0000 gloss:27.1964 gloss1-lgP:0.6935 gloss2-gQs:13.0114 gloss3-dQs:13.2067 gloss4-tgtQ:12.9984 dloss:5.4060 exploreP:0.9940
Episode:2725 Steps:26 meanR:23.5800 R:27.0000 gloss:26.1524 gloss1-lgP:0.6948 gloss2-gQs:12.3329 gloss3-dQs:12.9638 gloss4-tgtQ:12.3351 dloss:13.7774 exploreP:0.9940
Episode:2726 Steps:10 meanR:23.3900 R:11.0000 gloss:25.7488 gloss1-lgP:0.6989 gloss2-gQs:12.2398 gloss3-dQs:12.2941 gloss4-tgtQ:12.3241 dloss:22.5162 exploreP:0.9940
Episode:2727 Steps:14 meanR:23.3700 R:15.0000 gloss:25.2685 gloss1-lgP:0.6972 gloss2-gQs:11.8836 gloss3-dQs:12.4316 gloss4-tgtQ:11.9287 dloss:27.9038 exploreP:0.9940
Episode:2728 Steps:12 meanR:23.3800 R:13.0000 gloss:26.9038 gloss1-lgP:0.6970 gloss2-gQs:13.1039 gloss3-dQs:12.3900 gloss4-tgtQ:13.1145 dloss:9.4646 exploreP:0.9940
Episode:2729 Steps:19 meanR:23.4700 R:20.0000 gloss:25.7802 gloss1-lgP:0.6933 gloss2-gQs:12.2890 gloss3-dQs:12.5820 gloss4-tgtQ:12.3096 dloss:6.2077 exploreP:0.9940
Episode

Episode:2774 Steps:24 meanR:22.3100 R:25.0000 gloss:28.6444 gloss1-lgP:0.6951 gloss2-gQs:14.1145 gloss3-dQs:12.9278 gloss4-tgtQ:14.1593 dloss:14.5072 exploreP:0.9940
Episode:2775 Steps:10 meanR:22.2400 R:11.0000 gloss:25.9431 gloss1-lgP:0.6920 gloss2-gQs:12.2206 gloss3-dQs:13.0266 gloss4-tgtQ:12.2417 dloss:6.1418 exploreP:0.9939
Episode:2776 Steps:13 meanR:21.9100 R:14.0000 gloss:27.1512 gloss1-lgP:0.6917 gloss2-gQs:13.0222 gloss3-dQs:13.2127 gloss4-tgtQ:13.0160 dloss:6.0688 exploreP:0.9939
Episode:2777 Steps:20 meanR:21.9400 R:21.0000 gloss:26.9300 gloss1-lgP:0.6951 gloss2-gQs:12.8660 gloss3-dQs:12.9746 gloss4-tgtQ:12.8985 dloss:4.2008 exploreP:0.9939
Episode:2778 Steps:20 meanR:21.9800 R:21.0000 gloss:27.3632 gloss1-lgP:0.6969 gloss2-gQs:13.1576 gloss3-dQs:12.9260 gloss4-tgtQ:13.1732 dloss:5.9710 exploreP:0.9939
Episode:2779 Steps:14 meanR:21.6600 R:15.0000 gloss:27.3808 gloss1-lgP:0.6931 gloss2-gQs:13.1834 gloss3-dQs:13.1195 gloss4-tgtQ:13.1993 dloss:5.2472 exploreP:0.9939
Episode:2

Episode:2825 Steps:11 meanR:20.4600 R:12.0000 gloss:26.0213 gloss1-lgP:0.6964 gloss2-gQs:12.1263 gloss3-dQs:13.1021 gloss4-tgtQ:12.1547 dloss:14.0964 exploreP:0.9938
Episode:2826 Steps:24 meanR:20.6000 R:25.0000 gloss:25.9081 gloss1-lgP:0.6946 gloss2-gQs:12.2895 gloss3-dQs:12.7186 gloss4-tgtQ:12.2896 dloss:8.4823 exploreP:0.9938
Episode:2827 Steps:16 meanR:20.6200 R:17.0000 gloss:26.1293 gloss1-lgP:0.6948 gloss2-gQs:12.4399 gloss3-dQs:12.6581 gloss4-tgtQ:12.5083 dloss:9.7063 exploreP:0.9938
Episode:2828 Steps:21 meanR:20.7100 R:22.0000 gloss:27.3421 gloss1-lgP:0.6980 gloss2-gQs:13.1790 gloss3-dQs:12.7765 gloss4-tgtQ:13.2152 dloss:9.1853 exploreP:0.9938
Episode:2829 Steps:12 meanR:20.6400 R:13.0000 gloss:26.3307 gloss1-lgP:0.6947 gloss2-gQs:12.6369 gloss3-dQs:12.6137 gloss4-tgtQ:12.6519 dloss:6.4716 exploreP:0.9938
Episode:2830 Steps:28 meanR:20.7300 R:29.0000 gloss:26.9611 gloss1-lgP:0.6969 gloss2-gQs:12.9429 gloss3-dQs:12.7670 gloss4-tgtQ:12.9726 dloss:8.2680 exploreP:0.9938
Episode:2

Episode:2876 Steps:18 meanR:21.8200 R:19.0000 gloss:27.2325 gloss1-lgP:0.6974 gloss2-gQs:13.1174 gloss3-dQs:12.7875 gloss4-tgtQ:13.1488 dloss:6.5905 exploreP:0.9937
Episode:2877 Steps:10 meanR:21.7200 R:11.0000 gloss:26.8007 gloss1-lgP:0.6972 gloss2-gQs:12.7657 gloss3-dQs:12.9038 gloss4-tgtQ:12.7717 dloss:5.1915 exploreP:0.9937
Episode:2878 Steps:17 meanR:21.6900 R:18.0000 gloss:27.4953 gloss1-lgP:0.6969 gloss2-gQs:13.2255 gloss3-dQs:12.9803 gloss4-tgtQ:13.2441 dloss:5.7841 exploreP:0.9937
Episode:2879 Steps:16 meanR:21.7100 R:17.0000 gloss:26.5041 gloss1-lgP:0.6948 gloss2-gQs:12.6506 gloss3-dQs:12.8308 gloss4-tgtQ:12.6679 dloss:7.9590 exploreP:0.9937
Episode:2880 Steps:18 meanR:21.7600 R:19.0000 gloss:27.1945 gloss1-lgP:0.6951 gloss2-gQs:13.0729 gloss3-dQs:12.9673 gloss4-tgtQ:13.0803 dloss:8.4857 exploreP:0.9937
Episode:2881 Steps:42 meanR:21.8700 R:43.0000 gloss:27.9583 gloss1-lgP:0.6960 gloss2-gQs:13.4258 gloss3-dQs:13.2808 gloss4-tgtQ:13.4514 dloss:9.0904 exploreP:0.9937
Episode:28

Episode:2926 Steps:25 meanR:21.4600 R:26.0000 gloss:26.1277 gloss1-lgP:0.7002 gloss2-gQs:12.2213 gloss3-dQs:12.7670 gloss4-tgtQ:12.2966 dloss:12.8558 exploreP:0.9936
Episode:2927 Steps:11 meanR:21.4100 R:12.0000 gloss:25.5124 gloss1-lgP:0.6947 gloss2-gQs:12.1250 gloss3-dQs:12.5096 gloss4-tgtQ:12.1051 dloss:19.7659 exploreP:0.9936
Episode:2928 Steps:10 meanR:21.3000 R:11.0000 gloss:27.3727 gloss1-lgP:0.6962 gloss2-gQs:13.3676 gloss3-dQs:12.6159 gloss4-tgtQ:13.3332 dloss:8.5076 exploreP:0.9936
Episode:2929 Steps:24 meanR:21.4200 R:25.0000 gloss:25.4211 gloss1-lgP:0.6943 gloss2-gQs:12.0348 gloss3-dQs:12.5005 gloss4-tgtQ:12.0782 dloss:6.7924 exploreP:0.9936
Episode:2930 Steps:21 meanR:21.3500 R:22.0000 gloss:26.0025 gloss1-lgP:0.6964 gloss2-gQs:12.5210 gloss3-dQs:12.2714 gloss4-tgtQ:12.5522 dloss:12.3576 exploreP:0.9936
Episode:2931 Steps:14 meanR:20.9800 R:15.0000 gloss:25.7398 gloss1-lgP:0.6926 gloss2-gQs:12.4110 gloss3-dQs:12.3568 gloss4-tgtQ:12.3988 dloss:10.6203 exploreP:0.9936
Episod

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [36]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.